Transcript1 - [link](https://www.youtube.com/watch?v=BDIRabVP24o) (ENGLISH SPEECH | ELON MUSK: Think Big & Dream Even Bigger)

Transcript2 - [link](https://www.englishspeecheschannel.com/english-speeches/elon-musk-speech/) (Elon Musk Speech: Future, A.I., and Mars)

Transcript3 - [link](https://www.youtube.com/watch?v=IgKWPdJWuBQ&t=21s) (Элон Маск: Элон Маск: Человек, создавший Tesla, SpaceX, SolarCity...)

Transcript4 - [link](https://www.youtube.com/watch?v=zIwLWfaAg-8) (Elon Musk: The future we're building -- and boring | TED)

Transcript5 - [link](https://www.youtube.com/watch?v=H7Uyfqi_TE8) (Making Humans a Multiplanetary Species)

In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
import os

In [2]:
def set_sentences(text):
    new = ''
    for i in range(len(text) - 1):
        if text[i] == '.' and text[i + 1] == ' ':
            new += '*'
        else:
            new += text[i]
    return new

In [8]:
def transform1():
    f = open('Transcript1.txt', 'r')
    
    text = f.read()
    sentences = []
    for sentence in text.split('.\n'):
        sentences.append(sentence.strip().replace('\n', ' '))
        
    f.close()
    
    return sentences

In [54]:
def parse():
    script_dir = os.path.dirname(os.path.realpath('chromedriver'))
    webdriver_path = os.path.join(script_dir, 'chromedriver')
    web = webdriver.Chrome(webdriver_path)
    
    link = 'https://www.englishspeecheschannel.com/english-speeches/elon-musk-speech/'
    web.get(link)
    sleep(5)
    
    read_more = web.find_element_by_css_selector('#readlink240196857')
    read_more.click()
    
    part1 = []
    header = web.find_element_by_css_selector('#qtcontents > div > div.col.s12.m12.l8 > div > div:nth-child(3) > div > div > div > div > div')
    for block in header.find_elements_by_css_selector('p')[3:6]: ## part1
        part1.append(block.text)

    part2 = web.find_element_by_css_selector('#read240196857').text.split('\n') ## part2
    text = part1 + part2

    sentences = []
    for block in text:
        if (block[-1] == '?' and len(block.split('.')) < 10) or (len(block.split('.')) < 6):
            continue

        block = set_sentences(block)
            
        for sentence in block.split('*'):
            sentences.append(sentence)

    sentences = list(filter(lambda x: ' – ' not in x, sentences))
    
    return sentences



In [55]:
def transform2(filename):
    f = open(filename, 'r')
    text = f.read().replace('\n\n', ' ').replace('(Laughter)', '').replace('(Applause)', '')
    f.close()

    first = text.split('EM:')[0]
    other = text.split('EM:')[1:]

    blocks = []
    blocks.append(first.split('Elon Musk:')[1].split('CA:')[0])

    for block in other:
        blocks.append(block.split('CA:')[0].replace('\n', ''))


    sentences = []
    for block in blocks:
        
        block = set_sentences(block)
        for sent in block.split('*'):
            sentences.append(sent)
            
    return sentences

In [56]:
def transform3():
    f = open('Transcript4.txt', 'r')

    text = f.read().replace('\n\n', ' ').replace('(Laughter)', '').replace('[APPLAUSE]', '')
    text = text.split('ELON MUSK:')[1]
    text = set_sentences(text)
    sentences = text.split('*')

    f.close()
    
    return sentences

In [57]:
speeches = []
speeches += transform1()
speeches += transform2('Transcript2.txt')
speeches += transform2('Transcript3.txt')
speeches += transform3()
speeches += parse()

In [70]:
speeches = list(filter(lambda x: len(x.split(' ')) > 3 and '--' not in x ,speeches))

In [116]:
df = pd.DataFrame({'Speech': speeches})
df['Speech'] = df['Speech'].apply(lambda x: x.replace('\\', ''))
df.to_csv('Elon_phrases.csv')